# STA 220 Data & Web Technologies for Data Analysis

### Lecture 15, 25/2/25, Interactive Visualization: Cartography

### Announcements 

- 

### Last week's topics
- Cartography
    - Chloropeth maps

<div>
    <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Carte_figurative_de_l%27instruction_populaire_de_la_France.jpg" width="300"/>
</center>
    </div>

<div>
    <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/e/e2/2019UKElectionMap.svg" width="300"/>
</center>
</div>

### Today's topics
- Cartography
    - Density maps

While gradual color schemes are most appropriate for chloropeth maps, they only allow to show a single feature. 

Another problem in chloropeth maps is that they do not accurately depict data over geographic space with the use of large blocks. 

Dasyncretic maps address this issue. They use auxiliary information to portray the data more accurately. They intersect geographical objects to filter out spatial information that does not contribute to the data. E.g., for California, you may consider [this](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-land-ownership-1/explore?location=38.944934%2C-120.161746%2C10.45) map. 

<div>
    <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/7/7e/Utah_Valley_dasymetric_map.png" width="1000"/>
</center>
</div>

Another popular map format are dot maps. Consider the following map from the 1931 Polish census. 

<div>
    <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/2/25/GUS_languages1931_Poland.jpg" width="500" />
        </center>
</div>

Lets give this map a modern touch! We will draw from [Paul Dziemielas](https://dziemiela.com/personal/interwar_poland.html) geographical boundaries and census results. 

In [ ]:
import requests
r = requests.get('https://www.dziemiela.com/personal/Interwar_Poland_1934_20142.json', headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
})
topoJSON = r.json() # this is in topoJSON format!

In [ ]:
topoJSON['objects']['Palatinates']['geometries']

In [ ]:
import folium
m = folium.Map(width=800, height=500, tiles = None,
               location=[53, 23], zoom_start=5)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite'
).add_to(m)

In [ ]:
#topoJSON['objects']['Districts']['geometries']#[0]['properties']['GEOID']

In [ ]:
folium.TopoJson(topoJSON,
    object_path='objects.Districts', 
    style_function=lambda feature: {
        "fillColor": None,
        "fillOpacity": 0.0,
        "color": "lightgray",
        "weight": 1,
        "dashArray": "1",
    }, overlay=True, control=False).add_to(m)

In [ ]:
folium.LayerControl().add_to(m)
m

Lets retrieve the census data from the same source.

In [ ]:
import requests, zipfile, io
 
r = requests.get('https://www.dziemiela.com/personal/Interwar_Poland_1934_20142.zip', 
                 headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
})
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("../data/polish_census")

`fiona` is a module to handle geopackages. We have data for the 1931 and 1921 census, and a school census of 1926. We are only interested in the 1931 census. 

In [ ]:
import fiona
fiona.listlayers('../data/polish_census/Interwar_Poland_1934.gpkg')

In [ ]:
import geopandas
districts = geopandas.read_file("../data/polish_census/Interwar_Poland_1934.gpkg", 
                                layer='Census_1931_Districts') 
districts.head(3)

In [ ]:
print('\n'.join(districts.columns))

Lets craft the data set that is used to plot dots. 

In [ ]:
import numpy as np
import pandas as pd
data = districts[['GEOID', 'POLISH', 'UKRAINIAN', 'RUSKI', 
                    'BELARUSIAN', 'LITHUANIAN', 'GERMAN', 'YIDDISH', 'HEBREW']].set_index('GEOID').dropna()
#data.head()

In [ ]:
data = data.apply(lambda x: np.floor(x / 10000).astype(int), axis = 1)

In [ ]:
data.head()

As for the UK election, choose colors for each category. 

In [ ]:
colorpicker = {lang: color for lang, color in zip(data.columns, 
    ['#de3e16', '#f7d914', '#1cbd87', '#36a334', '#b569e0', '#64a8ed', '#b9d676', '#f781b2'])}

In [ ]:
import matplotlib.pyplot as plt

y = [0, 1]
x = [1, 1]

fig, axes = plt.subplots(ncols=4,nrows=2, sharex=True, sharey=True,
                         figsize=(5,2), subplot_kw={'xticks': [], 'yticks': []})

for ax, key in zip(axes.flat, colorpicker.keys()):
    ax.plot(x, y)
    ax.fill_betweenx(y, 0, 1, facecolor=colorpicker[key])
    ax.set_xlim(0, 0.1)
    ax.set_ylim(0, 1)
    ax.set_title(str(key))

plt.tight_layout()
plt.show()

Even though topoJSON is a more economical data format, we want to generate random points in each geometric object. To do so, we need to recast the topoJSON in to geoJSON format. 

In [ ]:
from pytopojson import feature
feature_ = feature.Feature()
geojson = feature_(topoJSON, 'Districts')

In [ ]:
geojson['features'][0] # navigate through... / do not print

In [ ]:
gdf = geopandas.GeoDataFrame.from_features(geojson['features'])
gdf.head(2)

In [ ]:
gdf['geometry'][2].bounds

In [ ]:
gdf['geometry'][2]

Random (on the cartesian plane) points are generated in each object. 

In [ ]:
import shapely, random
def generate_random_points(number, GEOID):

    # Select list entry of given object
    polygon = gdf[gdf['GEOID'] == GEOID]['geometry']#[0]
    # Extract bounding box (extent) from the GeoDataFrame
    minx, miny, maxx, maxy = polygon.bounds.squeeze()
    
    # Generate random points within the bounding box
    random_points = []
    while len(random_points) < number:
        random_point = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        # Check if the point is inside any of the polygons
        if all(random_point.intersects(polygon)):
            random_points.append(random_point)

    return geopandas.GeoDataFrame(geometry=random_points)['geometry']

In [ ]:
generate_random_points(2, 'P1613')

In [ ]:
generate_random_points(1, 'P1613')[0]

Finally, lets add the dots to the map. 

In [ ]:
m = folium.Map(width=800, height=500, tiles = None,
               location=[53, 23], zoom_start=5)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite'
).add_to(m)

folium.TopoJson(topoJSON,
    object_path='objects.Districts', 
    style_function=lambda feature: {
        "fillColor": None,
        "fillOpacity": 0.0,
        "color": "lightgray",
        "weight": 1,
        "dashArray": "1",
    }, overlay=True, control=False).add_to(m)

In [ ]:
for lang, countsvector in dict(data).items():
    color = colorpicker[lang]
    fg = folium.FeatureGroup(name=lang).add_to(m)
    for GEOID, counts in dict(countsvector).items(): 
        for point in generate_random_points(counts, GEOID): 
            folium.CircleMarker(location=[point.y, point.x], 
                    stroke=False,
                    fill=True,
                    color=color, 
                    fill_opacity=1,
                    radius=2).add_to(fg)

In [ ]:
folium.LayerControl(collapsed = False).add_to(m)
m 

<div>
    <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/2/25/GUS_languages1931_Poland.jpg" width="500"/>
</center>
    </div>

So why did the Polish census agency decide for a dot map? Lets create a plurality map. 

In [ ]:
district_colors = districts[['GEOID', 'POLISH', 'UKRAINIAN', 'RUSKI', 
                            'BELARUSIAN', 'LITHUANIAN', 'GERMAN', 'YIDDISH', 'HEBREW']].set_index('GEOID').dropna().idxmax(axis=1)
district_colors

In [ ]:
colorpicker

Lets add the palatinates as well. 

In [ ]:
palatinates = geopandas.read_file("../data/polish_census/Interwar_Poland_1934.gpkg", layer='Census_1931_Palatinates')
palatinate_colors = palatinates[['GEOID', 'POLISH', 'UKRAINIAN', 'RUSKI', 
                                 'BELARUSIAN', 'LITHUANIAN', 'GERMAN', 'YIDDISH', 'HEBREW']].set_index('GEOID').dropna().idxmax(axis=1) 

In [ ]:
m = folium.Map(width=800, height=500, tiles = None,
               location=[53, 23], zoom_start=5)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite'
).add_to(base_map)
base_map.add_to(m)

folium.TopoJson(topoJSON,
    name = "Districts",
    object_path='objects.Districts', 
    style_function=lambda feature: {
        "fillColor": colorpicker[district_colors[feature['properties']['GEOID']]],
        "fillOpacity": 0.8,
        "color": "lightgray",
        "weight": 1,
        "dashArray": "1",
    }, overlay=False).add_to(m)

folium.TopoJson(topoJSON,
    name = 'Palatinates',
    object_path='objects.Palatinates', 
    style_function=lambda feature: {
        "fillColor": colorpicker[palatinate_colors[feature['properties']['GEOID']]],
        "fillOpacity": 0.8,
        "color": "lightgray",
        "weight": 1, 
        "dashArray": "1",
    }, overlay=False).add_to(m)

In [ ]:
folium.LayerControl(collapsed = False).add_to(m)
m

The actual map from the census did only consider the categories 'Polish' or 'Other'. 

In [ ]:
district_colors = districts[['GEOID', 'POLISH', 'UKRAINIAN', 'RUSKI', 
                            'BELARUSIAN', 'LITHUANIAN', 'GERMAN', 'YIDDISH', 'HEBREW']].set_index('GEOID').dropna()

district_colors = pd.DataFrame({"POLISH": district_colors['POLISH'], 
                                "OTHER": district_colors.drop('POLISH', axis=1).sum(axis=1)}).idxmax(axis=1)

In [ ]:
palatinates = geopandas.read_file("../data/polish_census/Interwar_Poland_1934.gpkg", layer='Census_1931_Palatinates')
palatinate_colors = palatinates[['GEOID', 'POLISH', 'UKRAINIAN', 'RUSKI', 
                                 'BELARUSIAN', 'LITHUANIAN', 'GERMAN', 'YIDDISH', 'HEBREW']].set_index('GEOID').dropna()

palatinate_colors = pd.DataFrame({"POLISH": palatinate_colors['POLISH'], 
                                  "OTHER": palatinate_colors.drop('POLISH', axis=1).sum(axis=1)}).idxmax(axis=1)

In [ ]:
colorpicker["OTHER"] = '#b9d676'

In [ ]:
m = folium.Map(width=800, height=500, tiles = None,
               location=[53, 23], zoom_start=5)
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite'
).add_to(base_map)
base_map.add_to(m)

folium.TopoJson(topoJSON,
    name = "Districts",
    object_path='objects.Districts', 
    style_function=lambda feature: {
        "fillColor": colorpicker[district_colors[feature['properties']['GEOID']]],
        "fillOpacity": 0.8,
        "color": "lightgray",
        "weight": 1,
        "dashArray": "1",
    }, overlay=False).add_to(m)

folium.TopoJson(topoJSON,
    name = 'Palatinates',
    object_path='objects.Palatinates', 
    style_function=lambda feature: {
        "fillColor": colorpicker[palatinate_colors[feature['properties']['GEOID']]],
        "fillOpacity": 0.8,
        "color": "lightgray",
        "weight": 1,
        "dashArray": "1",
    }, overlay=False).add_to(m)

In [ ]:
folium.LayerControl(collapsed = False).add_to(m)
m 